In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import SimpleRNN, LSTM, Dropout, Flatten, Dense
import tensorflow as tf
from tensorflow.keras.models import Sequential

In [2]:
# This code imports the data you generated before for replicating.

stock_prices = pd.read_csv('stock_prices.csv')
arima_predictions_T = pd.read_csv('arima_predictions_T')

diff_T = stock_prices['T'].diff().dropna()

split = int(len(diff_T.values)*0.95)
diff_train_T = diff_T.iloc[:split]
diff_test_T = diff_T.iloc[split:]

In [3]:
n_steps = 10
n_features = 1

In [4]:
# This code imports the data you generated before for replicating.

def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
# This code is imported from the previous milestone for replicating.

X_diff_train_T, y_diff_train_T = split_sequence(diff_train_T, n_steps)
X_diff_train_T = X_diff_train_T.reshape((X_diff_train_T.shape[0],
                                         X_diff_train_T.shape[1], n_features))

X_diff_test_T, y_diff_test_T = split_sequence(diff_test_T.values, n_steps)
X_diff_test_T = X_diff_test_T.reshape((X_diff_test_T.shape[0],
                                       X_diff_test_T.shape[1], n_features))

In [6]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [7]:
def RNN_part():
    model = Sequential()
    model.add(SimpleRNN(128,
              input_shape=(n_steps, n_features),
              return_sequences=True))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer = 'adam' , loss='mean_squared_error', metrics=['mse'])
    return model

In [8]:
RNN_model = RNN_part()
history_RNN = RNN_model.fit(X_diff_train_T, y_diff_train_T,
                            batch_size=200,
                            epochs=200,
                            validation_split=0.2, 
                            callbacks=[callback])

Epoch 1/200
2/2 [==============================] - 1s 194ms/step - loss: 0.4194 - mse: 0.4194 - val_loss: 0.1326 - val_mse: 0.1326


In [9]:
# This code is imported from the previous milestone for replicating.


start = X_diff_test_T[X_diff_test_T.shape[0]-1]
T_input = start
T_input = T_input.reshape((1, n_steps, n_features))

In [10]:
# This code is imported from the previous milestone for replicating.
predictions_T = []


for i in range(len(arima_predictions_T)):
    T_input = T_input.reshape((1, n_steps, n_features))
    yhat = RNN_model.predict(T_input, verbose=0)
    T_input = np.append(T_input, yhat)
    T_input = T_input[1:]
    predictions_T.append(yhat)

In [12]:
def rmse(y,yhat):
    return np.sqrt(mean_squared_error(y,yhat))

In [13]:
print('RMSE of AT&T for RNN model {:.4f}'\
      .format(rmse(diff_test_T, np.array(predictions_T).flatten())))

RMSE of AT&T for RNN model 0.4566
